In [1]:
import pandas as pd # se importa el modulo pandas para manejo de dataframes.
import glob         # Importa libreria para manejo de archivos con cierto pathname definido
import lasio        # Permite ejecutar unas conversión sobre archivos con extensión .las

# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------

archivos_las = glob.glob('*.las') # genera una lista con todos los .las de la carpeta
frames = []

# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------

for n in range(0, len(archivos_las)): # loop que permite recorrer todos los archivos
    df = lasio.read(archivos_las[n]).df() # genera un dataframe por cada archivo
    df.reset_index(level=0, inplace=True)
    x = 0 # se establece el valor cero de inicio a la variable x.
    mV = 20 # se define el valor de amplitud para el cual se considera el tope de cemento.
    
    paso = [] # se define la lista paso vacía (Sería más reocmendable generar una Serie de pandas)

    for o in range(0, len(df.DEPT)-1): # se hace un loop sobre la lista para generar la columna paso.
        paso.append(df.DEPT[o+1] - df.DEPT[o])

    paso.append(paso[len(paso)-1]) # se genera el último valor de la lista para que paso tenga la misma cantidad de items que las otras columnas

    s = pd.Series(paso)

    df2 = pd.concat([df,s], axis=1) # agrega la Serie: paso, al dataframe df y se reemplaza el nombre.

    # df2.DEPT es la lista DEPT del dataframe df, en realidad serie de pandas DEPT
    # df2.CBL (AMP3FT) es la lista CBL del dataframe df, en realidad serie de pandas CBL.

    for i in range(0, len(df2.DEPT)): # empiezo un loop de cero a la longitud de la lista CBL.
        if df2.AMP3FT[i] <= mV and x < 3: # genero dos condiciones x < 3 (este valor puede cambiar), y el CBL menor o igual al definido (este valor también puede cambiar).
            x = x + df2[0][i] # sumo un paso al valor de la variable x cada que ve que se cumple la condición anterior.
            y = i # la variable y adopta el valor más actual de i.
            # print(y, x), para verificar cómo avanza el loop y comprobar errores.
        elif df2.AMP3FT[i] > mV and x < 3: # genero otras dos condiciones por si no se cumplen las anteriores. En este caso si el CBL es mayor al estabelcido, x vuelve a cero.
            x = 0 # si el valor del CBL es mayor al establecido como bueno, la cuenta vuelve a cero.
            # y = i, para verificar cómo avanza el loop y comprobar errores.
            # print(y, x), para verificar cómo avanza el loop y comprobar errores.
        else:
            break # si no se cumplen ninguna de las condiciones anteriores, el loop rompe.

    # Una vez que el loop rompe, tengo el valor de y = i de la iteración en la cual se cumplen x (en este caso 3) metros.
    # Por lo tanto x será 3 o más, i será el nñumero de iteración que cumple con x = 3 o más.

    z = y - int(x / ((df2[0][i]) - 1)) # z es el valor donde la función comienza a cumplirse.
    # toc = df2.DEPT[z] es el tope de cemento definido.

    df_new = df2[z:len(df2)] # genero un dataframe en el que se eliminan los valores superiores al toc: tope de cemento.
    df_new # visualizo el dataframe generado.

    key = ['mean', 'max', 'min', 'q5', 'q10', 'q15', 'q20', 'q25', 'q30', 'q35', 'q40', 'q45', 'q50', 'q55', 'q60', 'q65', 'q70', 'q75', 'q80', 'q85', 'q90', 'q95'] 
    # lista con cuantiles generados
    values = [] # se genera una lista vacia que se ira llenando a medida que el loop avance
    values.append(df_new['AMP3FT'].mean()) # agrega la media a la lista values.
    values.append(df_new['AMP3FT'].max()) # agrega el valor máximo a la lista values.
    values.append(df_new['AMP3FT'].min()) # agrega el valor mínimo a la lista values.

    for q in range(5,100,5): # loop desde 5 a 100 con paso 5 para los distintos cuantiles.
        values.append(df_new['AMP3FT'].quantile(q/100)) # función que agrega el valor calculado del cuantil a la lista values.

    df_final = pd.DataFrame(values, index=key, columns=[archivos_las[n]]) # se genera un dataframe con los valores de la lista values y los indices de la lista key. Ver cómo trasponer.
    df_final_2 = df_final.T # se muestra el dataframe.
    
    df_final_2

In [3]:
df_final_2

,mean,max,min,q5,q10,q15,q20,q25,q30,q35,...,q50,q55,q60,q65,q70,q75,q80,q85,q90,q95
YPF.SC.CS-2129_CBL-VDL_11-DIC-17.las,26.425491,73.0675,0.2301,0.71333,0.9946,1.38774,2.0727,3.3361,5.6329,8.9247,...,23.7078,28.9896,34.9981,40.1382,43.8975,47.5983,50.39074,52.8141,55.2495,59.0158
